# Introduction to the Planner

The Planner is one of the fundamental concepts of the Semantic Kernel. It makes use of the collection of plugins that have been registered to the kernel and using AI, will formulate a plan to execute a given ask.

Read more about it [here](https://aka.ms/sk/concepts/planner).

In [11]:
#r "nuget: Microsoft.SemanticKernel, 1.11.1"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.11.1-preview"

#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.0.1 Microsoft.SemanticKernel.Planners.Handlebars, 1.0.1-preview

### Setting Up Handlebars Planner
Handlebars Planner is located in the `Microsoft.SemanticKernel.Planning.Handlebars` package.

In [2]:
using Microsoft.SemanticKernel.Planning.Handlebars;

#pragma warning disable SKEXP0060

var planner = new HandlebarsPlanner();

### Providing plugins to the planner
The planner needs to know what plugins are available to it. Here we'll import the `SummarizePlugin` and `WriterPlugin` we have defined on disk.

In [3]:
var pluginsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "..", "..", "prompt_template_samples");

kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "SummarizePlugin"));
kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "WriterPlugin"));

Define your ASK. What do you want the Kernel to do?

In [4]:
#pragma warning disable SKEXP0060

var ask = "Tomorrow is Valentine's day. I need to come up with a few date ideas. My significant other likes poems so write them in the form of a poem.";
var originalPlan = await planner.CreatePlanAsync(kernel, ask);

Console.WriteLine("Original plan:\n");
Console.WriteLine(originalPlan);

Original plan:

{{!-- Step 1: Set the scenario for the poem --}}
{{set "scenario" "Valentine's Day"}}

{{!-- Step 2: Call the WriterPlugin-ShortPoem helper --}}
{{set "poem" (WriterPlugin-ShortPoem input=scenario)}}

{{!-- Step 3: Print the poem to the screen --}}
{{json poem}}


As you can see in the above plan, the Planner has taken the user's ask and converted it into a Plan object detailing how the AI would go about solving this task.

It makes use of the plugins that the Kernel has available to it and determines which functions to call in order to fulfill the user's ask.

Let's also define an inline plugin and have it be available to the Planner.
Be sure to give it a function name and plugin name.

In [5]:
string skPrompt = """
{{$input}}

Rewrite the above in the style of Shakespeare.
""";

var executionSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 2000,
    Temperature = 0.7,
    TopP = 0.5
};

var shakespeareFunction = kernel.CreateFunctionFromPrompt(skPrompt, executionSettings, "Shakespeare");

Let's update our ask using this new plugin.

In [6]:
#pragma warning disable SKEXP0060

var ask = @"Tomorrow is Valentine's day. I need to come up with a few date ideas.
She likes Shakespeare so write using his style. Write them in the form of a poem.";

var newPlan = await planner.CreatePlanAsync(kernel, ask);

Console.WriteLine("Updated plan:\n");
Console.WriteLine(newPlan);

Updated plan:

{{!-- Step 1: Create an array of date ideas --}}
{{set "dateIdeas" (array "Visit a Shakespearean play" "Have a picnic in the park and recite sonnets" "Take a romantic walk near the river" "Write love letters to each other" "Explore a Shakespearean-inspired art exhibition" "Enjoy a candlelit dinner and quote Romeo and Juliet" "Watch a movie adaptation of a Shakespearean play" "Attend a Shakespearean-themed festival" "Go star-gazing and recite Shakespearean sonnets" "Visit a historic location associated with Shakespeare")}}

{{!-- Step 2: Create an empty array to hold the lines of the poem --}}
{{set "poemLines" (array)}}

{{!-- Step 3: Loop through each date idea --}}
{{#each dateIdeas}}
  {{!-- Step 4: Capitalize the first letter of the date idea --}}
  {{set "capitalizedIdea" (capitalize this)}}

  {{!-- Step 5: Create a line of the poem using the date idea --}}
  {{set "poemLine" (concat "Forsooth, let us " capitalizedIdea " my dear")}}

  {{!-- Step 6: Add the line to

### Executing the plans

Now that we have different plans, let's try to execute them! The Kernel can execute the plan using RunAsync.

In [7]:
#pragma warning disable SKEXP0060

var originalPlanResult = await originalPlan.InvokeAsync(kernel, new KernelArguments());

Console.WriteLine("Original Plan results:\n");
Console.WriteLine(Utils.WordWrap(originalPlanResult.ToString(), 100));

Original Plan results:

Roses are red, violets are blue,
Valentine's Day, what shall we do?
We'll exchange gifts, and eat
lots of sweets,
But let's not forget, love's not just for treats!

Cupid's arrows flying, love is in
the air,
But beware of



Now lets execute and print the new plan:

In [12]:
#pragma warning disable SKEXP0060

var newPlanResult = await newPlan.InvokeAsync(kernel, new KernelArguments());

Console.WriteLine("New Plan results:\n");
Console.WriteLine(newPlanResult);

Error: HandlebarsDotNet.HandlebarsCompilerException: An unhandled exception occurred while trying to compile the template
 ---> HandlebarsDotNet.HandlebarsCompilerException: An unhandled exception occurred while trying to compile the template
 ---> HandlebarsDotNet.HandlebarsCompilerException: An unhandled exception occurred while trying to compile the template
 ---> HandlebarsDotNet.HandlebarsCompilerException: An unhandled exception occurred while trying to compile the template
 ---> HandlebarsDotNet.HandlebarsCompilerException: Sub-expression does not contain a converted MethodCall expression
   at HandlebarsDotNet.Compiler.SubExpressionVisitor.VisitSubExpression(SubExpressionExpression subex)
   at HandlebarsDotNet.Compiler.FunctionBuilder.Reduce(Expression expression, CompilationContext context, IReadOnlyList`1& decorators)
   at HandlebarsDotNet.Compiler.FunctionBinderHelpers.<>c__DisplayClass2_0.<CreateArguments>b__1(Expression o)
   at System.Linq.Enumerable.SelectEnume